# 1. Overview

- RDD is a collection of elements partitioned across nodes
- RDD can be persisted in memory
- RDD can recover from node failure

# 2. Creating a RDD

## 2.1. Parallelized  Collections

```python
    data = [1, 2, 3, 4, 5]
    distData = sc.parallelize(data)
```

To specify the number of partitions, we can do it like this

```python
    distData = sc.parallelize(data, 10)
```

## 2.2. External Datasets

- All of Spark file-based input methods support:
    - Directories
    - Compressed files
    - Wildcards
    
- `textFile()` method also has an optional argument for controlling the number of partitions of the file (128MB by HDFS default)

- `wholeTextFiles()` and `textFile()` are different in outputs when reading a directory:
    - `wholeTextFiles()`: return (key, value) pairs of (filename, content)
    - `textFile()`: return lines in each file, no filename specification

- RDD can be saved in pickled Python objects: `saveAsPickleFile()`

# 3. RDD Operations

- 2 types of operations:
    - transformation: create a new dataset from an existing one
    - action: return a value to driver program after running a computation on the dataset

- All transformations in Spark are *lazy*, results are not computed right away, they are remembered.

- Transformations are computed when an actions required a result to be returned to the drive program

- Can use caching for faster access

Example:

```python
    lines = sc.textFile("data.txt")
    lineLengths = lines.map(lambda s: len(s))
    totalLength = lineLengths.reduce(lambda a, b: a + b)
```

- The first and second lines are not executed right away, not until the third line, which is an action

- At that point, Spark breaks the computation into tasks to run on separate machines

To use `lineLengths` again later:

```python
    # persist() is more customized than cache()
    # Allow choosing either RAM or Disks as memory
    lineLengths.persist()
```

# 4. Using a global parameter for keeping track of the progress of a Spark job

- Just use Accumulators, don't think other solutions just yet

# 5. Printing elements in an RDD

- Load the RDD to the driver first, either using `collect()` or `take()`

# 6. Transformation

| **Transformation** | **Meaning** |
|--|--|
| **map**(*func*) | Return a new distributed dataset by passing each element of the source through the *func*|
| **filter**(*func*) | Return a new dataset by selecting the elements of the source on which *func* returns True |
| **flatMap**(*func*) | Like **map**(*func*) but return a sequence, rather than a single item |
| **mapPartitions**(*func*) | Like **map**(*func*) but the *func* should be Iterator &rarr; Iterator, since this transformation runs on partitions |
| **mapPartitionsWithIndex**(*func*) | Like **mapPartitions**(*func*) but the *func* should have another parameter to specify which index of the partition to transform |
| **sample**(*withReplacement*, *fraction*, *seed*) | Return a sample of the RDD |
| **union**(*otherDataset*) | Return a union of the source RDD and the one in the parameter |
| **intersection**(*otherDataset*) | Return a interection of the source RDD and the one in the parameter |
| **distinct**([*numPartitions*]) | Return a new dataset that contains the distinct elements of the source dataset |
| **groupByKey**([*numPartitions*]) | Take in (key, value), return (key, values) in groups based on key |
| **reduceByKey**(*func*, [*numPartitions*]) | Take in (key, value), return (key, agg(values)) in groups based on key |
| **aggregateByKey**(*func*, [*numPartitions*]) | Take in (key, value), return (key, agg(values)) in groups based on key |